In [ ]:
import requests

def read_dataset_from_url(url):
    response = requests.get(url)

    # Ensure request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {response.status_code}")

    lines = response.text.strip().split("\n")  # Split text by lines

    # Read first line: total flights (N) and total pairings (M)
    flight_count, pairing_count = map(int, lines[0].split())

    # Read crew pairings
    pairings = []
    for line in lines[1:]:  # Skip first line
        data = list(map(int, line.split()))
        cost = data[0]   # First number is cost
        flights = data[2:]  # Flights covered (skip second number)
        pairings.append([cost, len(flights)] + flights)  # Store in list format

    return flight_count, pairing_count, pairings

# Example usage
url = "http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/sppnw41.txt"
flight_count, pairing_count, pairings = read_dataset_from_url(url)

# Print parsed data
print("Total Flights:", flight_count)
print("Total Pairings:", pairing_count)
print("First 5 Pairings:", pairings[:5])  # Display first 5 pairings


Total Flights: 17
Total Pairings: 197
First 5 Pairings: [[2259, 5, 1, 3, 4, 8, 10], [3309, 4, 1, 3, 4, 11], [4497, 3, 1, 3, 4], [4965, 4, 1, 4, 9, 11], [5961, 3, 1, 4, 9]]


In [ ]:
# import numpy as np

# # Fitness function: Evaluates the cost of a solution
# def compute_cost(solution, pairings, flight_count, penalty=1000):
#     total_cost = 0
#     covered_flights = set()

#     for i in range(len(solution)):
#         if solution[i] == 1:
#             total_cost += pairings[i][0]  # Add pairing cost
#             covered_flights.update(pairings[i][2:])  # Add covered flights

#     # Apply penalty if not all flights are covered
#     missing_flights = flight_count - len(covered_flights)
#     if missing_flights > 0:
#         total_cost += penalty * missing_flights  # Large penalty for missing flights

#     return total_cost

# # Crossover (One-point crossover)
# def crossover(parent1, parent2):
#     point = np.random.randint(1, len(parent1))
#     child1 = np.concatenate((parent1[:point], parent2[point:]))
#     child2 = np.concatenate((parent2[:point], parent1[point:]))
#     return child1, child2

# # Mutation (Bit-flip mutation)
# def mutation(solution, mutation_rate=0.1):
#     mutated_solution = solution.copy()
#     for i in range(len(solution)):
#         if np.random.rand() < mutation_rate:
#             mutated_solution[i] = 1 - mutated_solution[i]  # Flip bit (0 ↔ 1)
#     return mutated_solution

# # Selection (Tournament selection)
# def tournament_selection(population, fitness, tournament_size=3):
#     selected_indices = np.random.choice(len(population), tournament_size, replace=False)
#     best_index = selected_indices[np.argmin(fitness[selected_indices])]  # Min cost is best
#     return population[best_index]

# # Standard Binary Genetic Algorithm (BGA)
# def bga(pairings, flight_count, population_size=100, generations=100, mutation_rate=0.1):
#     # Initialize random population
#     population = np.random.choice([0, 1], size=(population_size, len(pairings)))
#     best_solution, best_cost = None, float('inf')

#     for gen in range(generations):
#         fitness = np.array([compute_cost(sol, pairings, flight_count) for sol in population])

#         # Store the best solution found
#         min_idx = np.argmin(fitness)
#         if fitness[min_idx] < best_cost:
#             best_solution, best_cost = population[min_idx], fitness[min_idx]

#         new_population = []
#         for _ in range(population_size // 2):  # Create pairs of offspring
#             parent1 = tournament_selection(population, fitness)
#             parent2 = tournament_selection(population, fitness)

#             child1, child2 = crossover(parent1, parent2)
#             child1 = mutation(child1, mutation_rate)
#             child2 = mutation(child2, mutation_rate)

#             new_population.extend([child1, child2])

#         population = np.array(new_population)

#     return best_solution, best_cost




In [ ]:
import numpy as np

# Fitness function: Evaluates the cost of a solution
def compute_cost(solution, pairings, flight_count, penalty=1000):
    total_cost = 0
    covered_flights = set()

    for i in range(len(solution)):
        if solution[i] == 1:
            total_cost += pairings[i][0]  # Add pairing cost
            covered_flights.update(pairings[i][2:])  # Add covered flights

    # Count how many flights are still missing
    all_flights = set(range(1, flight_count + 1))
    missing_flights = len(all_flights - covered_flights)

    if missing_flights > 0:
        total_cost += penalty * missing_flights  # Large penalty for missing flights

    return total_cost

# Crossover (One-point crossover)
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# Mutation (Bit-flip mutation)
def mutation(solution, mutation_rate=0.1):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if np.random.rand() < mutation_rate:
            mutated_solution[i] = 1 - mutated_solution[i]  # Flip bit (0 ↔ 1)
    return mutated_solution

# Selection (Tournament selection)
def tournament_selection(population, fitness, tournament_size=3):
    selected_indices = np.random.choice(len(population), tournament_size, replace=False)
    best_index = selected_indices[np.argmin(fitness[selected_indices])]
    return population[best_index].copy()

# Standard Binary Genetic Algorithm (BGA) with fixes
def bga(pairings, flight_count, population_size=100, generations=1000, mutation_rate=0.1):
    # Initialize a more diverse population
    population = np.random.choice([0, 1], size=(population_size, len(pairings)), p=[0.2, 0.8])

    best_solution, best_cost = None, float('inf')

    for gen in range(generations):
        fitness = np.array([compute_cost(sol, pairings, flight_count) for sol in population])

        # Store the best solution found
        min_idx = np.argmin(fitness)
        if fitness[min_idx] < best_cost:
            best_solution, best_cost = population[min_idx].copy(), fitness[min_idx]

        new_population = [best_solution.copy()]  # Elitism: Keep the best solution

        while len(new_population) < population_size:
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)

            child1, child2 = crossover(parent1, parent2)
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = np.array(new_population[:population_size])  # Ensure correct population size

    return best_solution, best_cost


In [ ]:
best_solution, best_cost =bga(pairings, flight_count)

print("Best Solution (Selected Pairings):", best_solution)
print("Best Cost:", best_cost)

Best Solution (Selected Pairings): [0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 0 1 1
 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 0 1 0 1 1 1 0 1 0]
Best Cost: 180378


In [ ]:
import numpy as np

# Compute cost function with adaptive penalty
def compute_cost(solution, pairings, flight_count, base_penalty=1000):
    total_cost = 0
    covered_flights = set()

    for i in range(len(solution)):
        if solution[i] == 1:
            total_cost += pairings[i][0]  # Add pairing cost
            covered_flights.update(pairings[i][2:])  # Add covered flights

    # Adaptive penalty for missing flights
    missing_flights = flight_count - len(covered_flights)
    penalty = base_penalty * (missing_flights / flight_count) if missing_flights > 0 else 0

    return total_cost + penalty

# Greedy initialization ensuring good starting solutions
def initialize_population(pairings, flight_count, population_size):
    population = np.zeros((population_size, len(pairings)), dtype=int)

    for i in range(population_size):
        covered_flights = set()
        available_pairings = sorted(pairings, key=lambda p: p[0] / (len(p) - 2))  # Cost-effective pairings

        while len(covered_flights) < flight_count:
            for j, p in enumerate(available_pairings):
                if np.random.rand() < 0.5:
                    population[i][j] = 1
                    covered_flights.update(p[2:])

    return population

# Two-point crossover for diversity
def crossover(parent1, parent2):
    point1, point2 = sorted(np.random.randint(1, len(parent1), size=2))
    child1 = np.concatenate((parent1[:point1], parent2[point1:point2], parent1[point2:]))
    child2 = np.concatenate((parent2[:point1], parent1[point1:point2], parent2[point2:]))
    return child1, child2

# Hybrid mutation: Remove a random pairing & repair solution (SA-like)
def hybrid_mutation(solution, pairings, flight_count):
    solution = solution.copy()
    selected_indices = np.where(solution == 1)[0]
    if len(selected_indices) > 0:
        remove_idx = np.random.choice(selected_indices)  # Remove a random pairing
        solution[remove_idx] = 0
    return repair_solution(solution, pairings, flight_count)  # Greedy repair

def roulette_wheel_selection(population, fitness, temp=10):
    fitness = fitness - np.min(fitness)  # Shift values to avoid overflow
    max_fitness = np.max(fitness)

    if max_fitness == 0:
        probabilities = np.ones(len(population)) / len(population)  # Uniform probabilities if all fitness values are equal
    else:
        scaled_fitness = np.exp(-fitness / (max_fitness + 1e-6))  # Avoid division by zero
        probabilities = scaled_fitness / np.sum(scaled_fitness)

    if np.any(np.isnan(probabilities)):  # If probabilities are NaN, use uniform selection
        probabilities = np.ones(len(population)) / len(population)

    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index].copy()

# Greedy repair to ensure all flights are covered
def repair_solution(solution, pairings, flight_count):
    covered_flights = set()
    for i in range(len(solution)):
        if solution[i] == 1:
            covered_flights.update(pairings[i][2:])

    missing_flights = set(range(1, flight_count + 1)) - covered_flights
    if missing_flights:
        available_pairings = sorted(
            pairings,
            key=lambda p: p[0] / (len(set(p[2:]) & missing_flights) if len(set(p[2:]) & missing_flights) > 0 else 1e6)
        )
        for j, p in enumerate(available_pairings):
            if solution[j] == 0 and missing_flights.intersection(p[2:]):
                solution[j] = 1
                covered_flights.update(p[2:])
                missing_flights -= set(p[2:])
                if not missing_flights:
                    break
    return solution

# Final improved BGA
def improved_bga(pairings, flight_count, population_size=100, generations=500, base_penalty=1000):
    population = initialize_population(pairings, flight_count, population_size)
    best_solution, best_cost = None, float('inf')

    for gen in range(generations):
        fitness = np.array([compute_cost(sol, pairings, flight_count, base_penalty) for sol in population])

        # Store the best solution found
        min_idx = np.argmin(fitness)
        if fitness[min_idx] < best_cost:
            best_solution, best_cost = population[min_idx].copy(), fitness[min_idx]

        new_population = [best_solution.copy()]  # Elitism: Keep best solution

        while len(new_population) < population_size:
            parent1 = roulette_wheel_selection(population, fitness)
            parent2 = roulette_wheel_selection(population, fitness)

            child1, child2 = crossover(parent1, parent2)
            child1 = hybrid_mutation(child1, pairings, flight_count)
            child2 = hybrid_mutation(child2, pairings, flight_count)

            # Ensure flight coverage
            child1 = repair_solution(child1, pairings, flight_count)
            child2 = repair_solution(child2, pairings, flight_count)

            new_population.extend([child1, child2])

        population = np.array(new_population[:population_size])  # Maintain correct population size

    return best_solution, best_cost

# Run the improved BGA
best_solution, best_cost = improved_bga(pairings, flight_count)

# Display results
print("Best Solution (Selected Pairings):", best_solution)
print("Best Cost:", best_cost)


Best Solution (Selected Pairings): [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]
Best Cost: 18439.823529411766


In [ ]:
import numpy as np

# Compute the cost function with penalties for too many pairings
def compute_cost(solution, pairings, flight_count, penalty=1000, pairing_penalty=100):
    total_cost = 0
    covered_flights = set()

    for i in range(len(solution)):
        if solution[i] == 1:
            total_cost += pairings[i][0]  # Add pairing cost
            covered_flights.update(pairings[i][2:])  # Add covered flights

    # Apply penalty if not all flights are covered
    missing_flights = flight_count - len(covered_flights)
    if missing_flights > 0:
        total_cost += penalty * missing_flights  # Large penalty for missing flights

    # Penalize excessive pairings to avoid unnecessary costs
    total_cost += pairing_penalty * sum(solution)

    return total_cost

# Greedy initialization for better starting solutions
def greedy_initialization(pairings, flight_count):
    solution = np.zeros(len(pairings))
    covered_flights = set()

    sorted_pairings = sorted(enumerate(pairings), key=lambda x: x[1][0] / len(set(x[1][2:])))

    for i, pairing in sorted_pairings:
        if covered_flights >= set(range(1, flight_count + 1)):
            break  # Stop when all flights are covered
        if not set(pairing[2:]).issubset(covered_flights):
            solution[i] = 1
            covered_flights.update(pairing[2:])

    return solution

# Two-point crossover
def crossover(parent1, parent2):
    point1, point2 = sorted(np.random.randint(1, len(parent1), 2))
    child1 = np.concatenate((parent1[:point1], parent2[point1:point2], parent1[point2:]))
    child2 = np.concatenate((parent2[:point1], parent1[point1:point2], parent2[point2:]))
    return child1, child2

# Adaptive Mutation
def mutation(solution, mutation_rate=0.2):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if np.random.rand() < mutation_rate:
            mutated_solution[i] = 1 - mutated_solution[i]  # Flip bit (0 ↔ 1)
    return mutated_solution

# Roulette Wheel Selection
def roulette_wheel_selection(population, fitness):
    probabilities = 1 / (1 + fitness)  # Lower cost = higher probability
    probabilities /= np.sum(probabilities)  # Normalize
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index].copy()

# Improved BGA
def improved_bga(pairings, flight_count, population_size=30, generations=1000, mutation_rate=0.2):
    population = np.array([greedy_initialization(pairings, flight_count) for _ in range(population_size)])
    best_solution, best_cost = None, float('inf')

    for gen in range(generations):
        fitness = np.array([compute_cost(sol, pairings, flight_count) for sol in population])

        # Store best solution (Elitism)
        min_idx = np.argmin(fitness)
        if fitness[min_idx] < best_cost:
            best_solution, best_cost = population[min_idx], fitness[min_idx]

        new_population = [best_solution.copy()]  # Keep the best solution

        for _ in range((population_size - 1) // 2):  # Generate offspring
            parent1 = roulette_wheel_selection(population, fitness)
            parent2 = roulette_wheel_selection(population, fitness)

            child1, child2 = crossover(parent1, parent2)
            mutation_rate = max(0.1, 0.5 * (1 - gen / generations))  # Adaptive mutation
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = np.array(new_population)

    return best_solution, best_cost

# Run Improved BGA
best_solution, best_cost = improved_bga(pairings, flight_count)

print("Best Solution (Selected Pairings):", best_solution)
print("Best Cost:", best_cost)


Best Solution (Selected Pairings): [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0.]
Best Cost: 17582.0


In [ ]:
import numpy as np

# Compute the cost function with penalty for uncovered flights
def compute_cost(solution, pairings, flight_count, penalty=1000):
    total_cost = 0
    covered_flights = set()

    for i in range(len(solution)):
        if solution[i] == 1:  # If pairing is selected
            total_cost += pairings[i][0]  # Add pairing cost
            covered_flights.update(pairings[i][2:])  # Add covered flights

    # Apply penalty if not all flights are covered
    missing_flights = flight_count - len(covered_flights)
    if missing_flights > 0:
        total_cost += penalty * missing_flights  # Large penalty for missing flights

    return total_cost

# One-Point Crossover
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# Bit-Flip Mutation
def mutation(solution, mutation_rate=0.1):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if np.random.rand() < mutation_rate:
            mutated_solution[i] = 1 - mutated_solution[i]  # Flip 0 ↔ 1
    return mutated_solution

# Tournament Selection
def tournament_selection(population, fitness, tournament_size=3):
    selected_indices = np.random.choice(len(population), tournament_size, replace=False)
    best_index = selected_indices[np.argmin(fitness[selected_indices])]  # Min cost is best
    return population[best_index]

# Standard BGA
def standard_bga(pairings, flight_count, population_size=30, generations=1000, mutation_rate=0.1):
    # Random Initialization
    population = np.random.choice([0, 1], size=(population_size, len(pairings)))
    best_solution, best_cost = None, float('inf')

    for gen in range(generations):
        fitness = np.array([compute_cost(sol, pairings, flight_count) for sol in population])

        # Store best solution found
        min_idx = np.argmin(fitness)
        if fitness[min_idx] < best_cost:
            best_solution, best_cost = population[min_idx], fitness[min_idx]

        new_population = []
        for _ in range(population_size // 2):  # Create pairs of offspring
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)

            child1, child2 = crossover(parent1, parent2)
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = np.array(new_population)

    return best_solution, best_cost

# Run Standard BGA
best_solution, best_cost = standard_bga(pairings, flight_count)
print("Best Solution (Selected Pairings):", best_solution)
print("Best Cost:", best_cost)


Best Solution (Selected Pairings): [0 1 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0
 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0
 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0
 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 1 0 0 0 1 0 0 1 1 0]
Best Cost: 228588
